In [1]:
import json
import pandas as pd
import requests

base_url = 'http://127.0.0.1:5000' ## Standard route for locally running Flask app


## Retrieve predictions
In this scenario, our API collects the data for scoring and sends us the result. Imagine this being analogous to a scheduled prediction workload where the application queries a database or other source for the most recent records and generates predictions.

In [2]:
r = requests.get(f'{base_url}/score')
df = pd.DataFrame(r.json()['prediction_data'])
df.head()

,Age,BMI,BloodPressure,DiabetesPedigreeFunction,Glucose,Insulin,Pregnancies,SkinThickness,prediction
0,22,32.0,75,0.396,79,0,1,30,0
1,43,35.0,82,0.233,95,180,1,25,0
2,31,29.8,0,0.501,91,0,6,0,0
3,25,19.6,66,0.334,95,38,1,13,0
4,63,29.8,70,0.207,158,0,5,0,1


## Streaming predictions
In this scenario, we `POST` data to the API and request that predictions be generated for these records. This is a pattern that you could leverage for a streaming prediction system.

In [3]:
## Sample records from our saved test data set
data = pd.read_csv('../diabetes_test_x.csv').sample(10)
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
26,8,84,74,31,0,38.3,0.457,39
12,2,81,72,15,76,30.1,0.547,25
125,1,106,70,28,135,34.2,0.142,22
126,3,191,68,15,130,30.9,0.299,34
133,0,137,68,14,148,24.8,0.143,21
77,4,171,72,0,0,43.6,0.479,26
25,3,74,68,28,45,29.7,0.293,23
191,2,119,0,0,0,19.6,0.832,72
71,2,90,60,0,0,23.5,0.191,25
112,3,123,100,35,240,57.3,0.880,22


In [4]:
## Convert records to JSON format
data = data.to_dict(orient = 'records')

## HTTP request headers to indicate what we're sending
headers = {
    'content-type': 'application/json'
}

## Formulate the POST request
r = requests.post(f'{base_url}/score', json = data, headers = headers)

## Parse the response data
response_data = json.loads(r.content.decode("utf-8"))['prediction_data']
df_result = pd.DataFrame(response_data)
df_result

,Age,BMI,BloodPressure,DiabetesPedigreeFunction,Glucose,Insulin,Pregnancies,SkinThickness,prediction
0,39,38.3,74,0.457,84,0,8,31,0
1,25,30.1,72,0.547,81,76,2,15,0
2,22,34.2,70,0.142,106,135,1,28,0
3,34,30.9,68,0.299,191,130,3,15,1
4,21,24.8,68,0.143,137,148,0,14,0
5,26,43.6,72,0.479,171,0,4,0,1
6,23,29.7,68,0.293,74,45,3,28,0
7,72,19.6,0,0.832,119,0,2,0,0
8,25,23.5,60,0.191,90,0,2,0,0
9,22,57.3,100,0.880,123,240,3,35,1
